In [1]:
import random
import numpy as np
from collections import deque
import pandas as pd 

# Parámetros

In [2]:
#Horizonte de simulacion (Turnos)
Turnos = 1
Reloj = 0
tiempo_aculatamiento = 70/60

In [3]:
#Se leen las caracteristicas de las palas
dfPalas = pd.read_excel('Informacion Palas.xlsx',  1)
InfoPalas = list(zip(dfPalas[dfPalas.columns[0]].values, dfPalas[dfPalas.columns[1]].values, dfPalas[dfPalas.columns[2]].values, \
                            dfPalas[dfPalas.columns[3]].values, dfPalas[dfPalas.columns[4]].values, \
                            dfPalas[dfPalas.columns[5]].values, dfPalas[dfPalas.columns[6]].values, \
                            dfPalas[dfPalas.columns[7]].values, dfPalas[dfPalas.columns[8]].values))

#Lista de nodos de mineral y esteril
Nodos_Mineral = ["PO1", "PO2", "PO3", "PO4", "PO5"]
Nodos_Esteril = ["PO6", "PO7", "PO8", "PO9", "PO10"]
Polvorazos = Nodos_Mineral + Nodos_Esteril


#Número de camiones disponibles
total_camiones = 8

#Lista de stocks/botaderos
#ID, espacios, ubicacion, tiempo_atencion ID=ubicacion?
SoB = ["S1", "S2"]
espacios = 5
tiempo_atencion_stock = 70/60 #segundos

#Se lee la informacion del chancador
dfChancador = pd.read_excel('Informacion Palas.xlsx',  0)
InfoChancador = list(zip(dfChancador[dfChancador.columns[0]].values, dfChancador[dfChancador.columns[1]].values, dfChancador[dfChancador.columns[2]].values, \
                            dfChancador[dfChancador.columns[3]].values, dfChancador[dfChancador.columns[4]].values, \
                            dfChancador[dfChancador.columns[5]].values, dfChancador[dfChancador.columns[6]].values, \
                            dfChancador[dfChancador.columns[7]].values, dfChancador[dfChancador.columns[8]].values))

# Dinamico

In [4]:
def dinamico(destinos, pala):
    return (random.choice(pala), random.choice(destinos))

# 1. Clases
## 1.1. Palas

In [5]:
class Pala:
    def __init__(self, ID, capacidad, ubicacion, media_recuperacion, media_falla, velocidad, tiempo_aculatado, min_carga, moda_carga, max_carga):
        self.ubicacion = ubicacion #Nodo en que esta ubicada la pala
        self.capacidad = capacidad #Capacidad nominal de excavacion
        self.ID = ID #ID pala
        self.fila = deque() #camiones en fila para se atendidos
        
        self.espacios_aculatamiento = [0, 0] #camiones aculatados 
        self.orden_llegada = deque() #orden de llegada al aculatamiento
        
        self.horas_eventos = {"Recuperacion":99999999, "Caida":9999999, "Inicio Movimiento":99999999, "Fin Movimiento":99999999, "Carga":999999} 
        self.proximo_evento = None #Nombre del evento mas proximo
        self.toneladas_sacadas = {} #Diccionario de tuplas con toneladas sacadas por posicion/nodo y ley
        self.TPH = {} #Diccionario con los resultados del PL
        
        self.media_recuperacion = media_recuperacion
        self.media_falla = media_falla
        self.velocidad = velocidad
        
        self.tiempo_aculatado = tiempo_aculatado
        
        self.min = min_carga
        self.moda = moda_carga
        self.max = max_carga
        
        self.hora_siguiente = 999999999
        self.disponibilidad = True
                     
    def caida(self, Reloj):#Completar comn PL y con Dinamico
        self.estado = 0
        #POR FUERA:
        #Se debe correr el PL y luego el dinámico para todos los camiones en la fila, y en camion hacia la pala
        #for cada camion if camion hacia pala.ID correr dinamico 
        self.horas_eventos["Caida"] = 99999999
        self.horas_eventos["Recuperacion"] = Reloj + self.tiempo_recuperacion()
        if self.horas_eventos["Recuperacion"] > self.horas_eventos["Inicio Movimiento"]:
            self.horas_eventos["Inicio Movimiento"] = self.horas_eventos["Recuperacion"] + 0.0001 #Postergo el movimiento hasta que se arregle la pala
        self.proximo_evento = "Recuperacion"
        return None
    
    def recuperacion(self, Reloj):
        self.estado = 1
        self.horas_eventos["Recuperacion"] = 99999999
        self.horas_eventos["Caida"] = Reloj + self.tiempo_falla()
        self.proximo_evento = min(self.horas_eventos, key=self.horas_eventos.get)
        self.hora_siguiente = self.horas_eventos[self.proximo_evento]
        return None
    
    def disponible(self):
        for espacio in self.espacios_aculatamiento:
            if espacio != 0:
                return True
        return False
    
  
    def fin_movimiento(self, Reloj, Lista_movimientos_agendados):
        pass
       
    def tiempo_recuperacion(self):
        return(np.random.exponential(self.media_recuperacion))
    
    def tiempo_falla(self):
        return(np.random.exponential(self.media_falla))
    
    def hora_proximo_evento(self):
        self.hora_siguiente = min(self.horas_eventos.values())
        return(self.hora_siguiente)
    
    def nombre_proximo_evento(self):
        self.proximo_evento = min(self.horas_eventos, key=self.horas_eventos.get)
        return(self.proximo_evento)
    
    def duracion_atencion(self):
        return(np.random.triangular(self.min, self.moda, self.max))
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        

## 1.2. Camiones

In [6]:
class Camion:
    def __init__(self, ID):
        self.ID = ID
        self.Nombre = "C"+str(ID)
        self.destino = "CS03"
        self.origen = "CS03"
        self.almuerzo = 0 #0: No almorzo, 1 primera colacion, 2 segunda colacion (solo aplica turno B)
        self.hora_siguiente = 0
        self.descargas = {} #Diccionario que almacena la historia de las descargas: key = (Origen, Destino)| value = toneladas
        self.carga = 0
        #self.aculatandose = False
        self.estado = "Descargando" #Viajando, Aculatandose, Cargando, Descargando, Almorzando, En fila
        self.OD = None
        
    def descargar(self, nuevo_destino, Reloj, TdV):
        self.toneladas = 0
        self.origen = self.destino
        self.destino = nuevo_destino
        if self.carga > 0:
            self.descargas[(self.origen, destino)] = self.carga 
        self.carga = 0
        self.estado = "Viajando"
        self.hora_siguiente = Reloj + TdV
        return None
    
    def cargar(self, toneladas_cargadas, nuevo_destino):
        self.origen = self.destino
        self.destino = nuevo_destino
        self.carga = toneladas_cargadas
        return None
    
    #def asignar_destino(self, Reloj, nuevo_destino, MTV):
    #    self.destino = nuevo_destino
    #    self.actualizar_estado("Viajando", Reloj + MTV[self.origen][self.destino])
    
    def actualizar_estado(self, Estado, Hora):
        self.estado = Estado 
        self.hora_siguiente = Hora
        
    def nuevo_viaje(self, Estado, Hora):
        self.estado = Estado 
        self.hora_siguiente = Hora
        self.origen = self.OD[0]
        self.destino = self.OD[1]
        
    def comenzar_almuerzo(self, destino, reloj, MTV, tiempo_colacion, almuerzo):
        self.origen = self.destino
        self.destino = destino
        self.estado = "Almorzando"
        self.hora_siguiente = Reloj + MTV[self.origen][self.destino] + tiempo_colacion
        self.almuerzo = almuerzo
        
    def __str__(self):
        return str(self.Nombre)
    def __unicode__(self):
        return str(self.Nombre)
    def __repr__(self):
        return str(self.Nombre)
        
        
        
        
        

## 1.3. Stock - Botadero

In [7]:
class Stock:
    
    def __init__(self, ID, espacios, ubicacion, tiempo_atencion, tiempo_aculatado):
        self.ID = ID
        self.fila = deque()
        self.espacios_aculatamiento = [0 for i in range(espacios)]
        self.ubicacion = ubicacion
        self.tiempo_atencion = tiempo_atencion
        self.hora_siguiente = 99999999
        self.tiempo_aculatado = tiempo_aculatado 
    
    def avanzar_fila(self):
        camion = self.fila.popleft()
        return camion
    
    def duracion_atencion(self):
        return(self.tiempo_atencion)
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        
   

## 1.4. Chancador

In [8]:
class Chancador:
    
    def __init__(self, ID, ubicacion, media_falla, media_recuperacion, tiempo_aculatado, min_carga, moda_carga, max_carga):
        self.ID = ID
        self.ubicacion = ubicacion
        self.espacios_aculatamiento = [0, 0]
        self.fila = deque()
        self.media_falla = media_falla
        self.media_recuperacion = media_recuperacion
        self.tiempo_aculatado = tiempo_aculatado
        self.min = min_carga
        self.moda = moda_carga
        self.max = max_carga
        self.hora_siguiente = 999999999
        
    def tiempo_recuperacion(self):
        return(np.random.exponential(self.media_recuperacion))
    
    def tiempo_falla(self):
        return(np.random.exponential(self.media_falla))
    
    def hora_proximo_evento(self):
        return(min(horas_eventos.values()))
    
    def duracion_atencion(self):
        return(np.random.triangular(self.min, self.moda, self.max))
    
    def __str__(self):
        return str(self.ID)
    def __unicode__(self):
        return str(self.ID)
    def __repr__(self):
        return str(self.ID)
        
    

# 2. Simulación
## 2.1. Inicialización 

In [9]:
#Se evalúa que palas estarán operativas y en que condición y se generan las instancias 
L_Entidades = []
#Esteril: 303
#Mineral: 293
D_Palas = {}
D_Entidades = {}
for pala in InfoPalas:
    u = np.random.uniform(0,1)
    
    if 0 <= u and u < pala[3]:
        pass
    
    elif pala[3] <= u and u < 1:#pala[4]
        D_Palas[pala[0]] = Pala(pala[0], "TONELADAS X CARGA", random.choice(Polvorazos), pala[2], pala[1], "velocidad", \
                                pala[8], pala[5], pala[6], pala[7])
        L_Entidades.append(D_Palas[pala[0]])
        D_Entidades[pala[0]] = D_Palas[pala[0]]
       
    else:
        #D_Palas[pala[0]] = Pala(pala[0], "TONELADAS X CARGA", random.choice(Polvorazos), pala[2], pala[1], "velocidad", \
        #                        pala[8], pala[5], pala[6], pala[7]) 
        #D_Palas[pala[0]].recuperacion(0)#Se inicializa el tiempo de la siguiente falla
        #print(pala[0], D_Palas[pala[0]].hora_siguiente)
        #L_Entidades.append(D_Palas[pala[0]])
        #D_Entidades[pala[0]] = D_Palas[pala[0]]
        pass
        
#Se inicializan los camiones
D_Camiones = {}
for i in range(total_camiones):
    D_Camiones[i] = Camion(i)
    L_Entidades.append(D_Camiones[i])
    D_Entidades[i] = D_Camiones[i]


    
#Se inicializan los stocks/botaderos
D_Stocks = {}
for stock in SoB:
    D_Stocks[stock] = Stock(stock, espacios, stock, tiempo_atencion_stock, tiempo_aculatamiento)
    L_Entidades.append(D_Stocks[stock])
    D_Entidades[stock] = D_Stocks[stock]
#print("Stocks y Botaderos :", D_Stocks)    

#Se inicializa el Chancador
D_Chancador ={}
for chancador in InfoChancador:
    D_Chancador[chancador[0]] = Chancador(chancador[0], "CS03", chancador[1], chancador[2], chancador[8], \
                                          chancador[5], chancador[6], chancador[7])
    L_Entidades.append(D_Chancador[chancador[0]])
    D_Entidades[chancador[0]] = D_Chancador[chancador[0]]
    
#print("Chancador: ", D_Chancador)


L_Entidades.sort(key=lambda x: x.hora_siguiente, reverse=False)


## 2.2. Matriz de tiempos de viaje

In [10]:
MTV = {}
Destinos = []
Palas = []

for destino1 in L_Entidades:
    if destino1.__class__.__name__ in ["Stock", "Chancador"]:
        Destinos.append(destino1.ID)
        MTV[destino1.ID] = {}
        for destino2 in L_Entidades:
            if destino2.__class__.__name__ in ["Stock", "Chancador"]:
                MTV[destino1.ID][destino2.ID] = random.uniform(20, 40)
                    
    if destino1.__class__.__name__ == "Pala":
        Palas.append(destino1.ID)
    #La Matriz de tiempos de viajes debe ser entre los polvorazos y los destinos    
    
for destino1 in Polvorazos:
    MTV[destino1] = {}
    for key in MTV.keys():
        MTV[key][destino1] = random.uniform(20, 40)
        MTV[destino1][key] = random.uniform(20, 40)
    
MTV["CASINO"] = {}
for key in MTV.keys():
    MTV[key]["CASINO"] = random.uniform(20, 40)
    MTV["CASINO"][key] = random.uniform(20, 40)
print(MTV)

{'S1': {'S1': 27.574099811151257, 'S2': 26.01386555773312, 'CS03': 33.68866202628135, 'PO1': 35.18521502236305, 'PO2': 22.99802445589508, 'PO3': 36.528420641738336, 'PO4': 34.867096288005285, 'PO5': 21.416272479048924, 'PO6': 33.11742819279944, 'PO7': 31.240628153089936, 'PO8': 34.81855744626655, 'PO9': 31.89517873256444, 'PO10': 37.97552907931241, 'CASINO': 36.95888738378207}, 'S2': {'S1': 33.05426726656376, 'S2': 36.20647048303157, 'CS03': 34.46005303354153, 'PO1': 27.512307253230688, 'PO2': 25.629078732190003, 'PO3': 24.137020438444647, 'PO4': 34.41587528358353, 'PO5': 23.76757286213065, 'PO6': 30.675627515616487, 'PO7': 33.47636072231024, 'PO8': 30.237771398925993, 'PO9': 36.0946559444201, 'PO10': 23.91241726319752, 'CASINO': 31.759184752746446}, 'CS03': {'S1': 22.163185995709846, 'S2': 39.28740339573632, 'CS03': 23.077705134274844, 'PO1': 22.61086458161483, 'PO2': 28.86555313410735, 'PO3': 31.616244495478142, 'PO4': 37.453769680793656, 'PO5': 25.48443943458112, 'PO6': 24.112651390

In [11]:
#Identificamos el siguiente evento que va a ocurrir}
TurnoA = True
TurnoB = False
NTurnos = 0

trucks = 0 
Entidad = L_Entidades[0]
Reloj = Entidad.hora_siguiente
while Reloj < 12*60:
    
    NTurnos = int(Reloj/(12*60))
    
    
    if int(Reloj/(12*60))%2 == 0:
        TurnoB = False
        TurnoA = True
    else:
        TurnoA = False
        TurnoB = True

    #Tomamos el primer elemento de la lista de Entidades
    Entidad = L_Entidades[0]
    Reloj = Entidad.hora_siguiente
    print('------------------------------------------------------------')
    print(Entidad, Reloj)
   
    #En caso de que la entidad sea un camion
    if Entidad.__class__.__name__ == "Camion":
        print(Entidad, Entidad.origen, Entidad.destino, Entidad.estado, Reloj, Entidad.almuerzo)
        if Entidad.estado == "Almorzando":

            Entidad.estado = "Descargando"
        
            print("El camion {} termino de almorzar.".format(Entidad))
            
        if Entidad.estado == "Descargando":#Salida de un stock, botadero o chancador
            #Comienzo de almuerzos
            
            #Almuerzo turno A
            MIT = NTurnos*60*12 #Minuto en que inicio turno
            if (TurnoA and 0 <= Entidad.ID and Entidad.ID <= 40 and Reloj >= (4.5*60 + MIT) and Entidad.almuerzo == 0):
                Destino = D_Entidades[Entidad.destino]
                print("El camion {} se va a almorzar desde {}".format(Entidad, Destino))
                print("La cola en el destino es {}".format(Destino.fila))
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo("CASINO", Reloj, MTV, 60, 1)
                print("La cola en el destino queda {}".format(Destino.fila))
                
                
            if (TurnoA and Entidad.ID > 40 and Reloj >= (6*60 + MIT) and Entidad.almuerzo ==0):
                Destino = D_Entidades[Entidad.destino]
                print("El camion {} se va a almorzar desde {}".format(Entidad, Destino))
                print("La cola en el destino es {}".format(Destino.fila))
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo("CASINO", Reloj, MTV, 60, 1)
                print("La cola en el destino queda {}".format(Destino.fila))
            
            #Primera colacion turno B
            if (TurnoB and 0 <= Entidad.ID and Entidad.ID <= 27 and Reloj >= (5*60 + MIT) and Entidad.almuerzo == 0):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 40, 1)
                
            if (TurnoB and 28 <= Entidad.ID and Entidad.ID <= 54 and Reloj >= (5.5*60 + MIT)and Entidad.almuerzo == 0):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 40, 1)
                
            if (TurnoB and 55 <= Entidad.ID and Reloj >= (4.5*60 + MIT) and Entidad.almuerzo == 0):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 40, 1)
                
            #Segunda colacion turno B
            if (TurnoB and 0 <= Entidad.ID and Entidad.ID <= 45 and Reloj >= (9*60 + MIT) and Entidad.almuerzo == 1):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 20, 2)
                
            if (TurnoB and 46 <= Entidad.ID and Reloj >= (9.5*60 + MIT) and Entidad.almuerzo == 1):
                Destino = D_Entidades[Entidad.destino]
                if len(Destino.fila) > 0:
                    #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Siguiente = Destino.fila.popleft()
                    Destino.espacios_aculatamiento[i] = Siguiente
                    Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                else:
                    i = Destino.espacios_aculatamiento.index(Entidad)
                    Destino.espacios_aculatamiento[i] = 0
                Entidad.comenzar_almuerzo(Entidad.origen, Reloj, MTV, 20, 2)
            
            #En caso contrario, hay que asignarle un nuevo destino corriendo el dinámico
            if Entidad.estado == "Descargando": #Posible fuente de error; probar -- (if Entidad.estado == "Descargando")
                #Pregunto si hay fila en el stock/botadero/chancador
                try:
                    Destino = D_Entidades[Entidad.destino]
                    if len(Destino.fila) > 0:
                        #Si hay fila avanzo al siguiente y lo ubico en el lugar del que esta saliendo
                        i = Destino.espacios_aculatamiento.index(Entidad)

                        print("La fila en {} es {}".format(Destino, Destino.fila))
                        print("El lugar de aculatado en {} es {}".format(Destino, Destino.espacios_aculatamiento))
                        Siguiente = Destino.fila.popleft()
                        Destino.espacios_aculatamiento[i] = Siguiente
                        Siguiente.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)

                        print("El camion {} sale y comienza el aculatado {}".format(Entidad, Siguiente))
                        print("La fila en {} quedo {}".format(Destino, Destino.fila))
                        print("El lugar de aculatado en {} quedo {}".format(Destino, Destino.espacios_aculatamiento))

                    elif Entidad in Destino.espacios_aculatamiento:
                        i = Destino.espacios_aculatamiento.index(Entidad)
                        print("No habia fila")
                        print("El lugar de aculatado en {} es {}".format(Destino, Destino.espacios_aculatamiento))
                        Destino.espacios_aculatamiento[i] = 0
                        print("El lugar de aculatado en {} quedo {}".format(Destino, Destino.espacios_aculatamiento))
                except:
                    pass

                nuevo_destino = dinamico(Destinos, Palas) #Decido hacia donde enviar el camion
                Entidad.OD = nuevo_destino #Guardo el resultado del dinamico - par OD
                print("El resultado del dinamico fue: {}".format(nuevo_destino))
                #Finalizo descarga, actualizo tiempo de sgte evento, y cambio estado a "Viajando", y origen
                Entidad.descargar(nuevo_destino[0], Reloj, MTV[Entidad.destino][D_Entidades[nuevo_destino[0]].ubicacion]) 
        
        elif Entidad.estado == "Viajando":
            Destino = D_Entidades[Entidad.destino]
            #print("Entre")
            if len(Destino.fila) > 0:#Reviso si hay fila en el destino
                Destino.fila.append(Entidad)#El camion se pone a la fila
                Entidad.actualizar_estado("En fila", Reloj*99)
                print("Camion {} se pone a la fila {}".format(Entidad, Destino.fila))
            
            if len(Destino.fila) == 0:
                #Se revisa la disponibilidad de espacios de aculatamiento
                if 0 in Destino.espacios_aculatamiento:
                    #Camion comienza a aculatarse
                    try:
                        print("No hay fila, {} se aculata {}".format(Entidad, Destino.orden_llegada))
                    except:
                        pass
                    Entidad.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                    #El camion se ubica en el lugar disponible
                    lugar = 0
                    for espacio in Destino.espacios_aculatamiento:
                        if espacio == 0:
                            Destino.espacios_aculatamiento[lugar] = Entidad
                            if Destino.__class__.__name__ == "Pala":
                                Destino.orden_llegada.append(Entidad)
                                
                                print("Camion aculatandose en {}:{}".format(Destino, Destino.espacios_aculatamiento))
                            break
                        lugar += 1
                else:
                    Destino.fila.append(Entidad)
                    Entidad.actualizar_estado("En fila", Reloj*99)  
                    print("Esta es la fila ahora: {}".format(Destino.fila))
        
        elif Entidad.estado == "Aculatandose":#Termina de aculatarse
            Destino = D_Entidades[Entidad.destino]
            
            if Destino.__class__.__name__ in ["Chancador", "Stock"]:
                Entidad.actualizar_estado("Descargando", Reloj + Destino.duracion_atencion())#Comienza descarga solo
                #Destino.hora_siguiente = Reloj + Destino.duracion_atencion()
                #Entidad.actualizar_estado("Descargando", Reloj*99999)
                
            if Destino.__class__.__name__ == "Pala":
                Entidad.actualizar_estado("Cargando", Reloj*99)
                Primero = Destino.orden_llegada.popleft()
                #Reviso si hay alguien mas siendo atendido:
                if Destino.disponibilidad and Entidad == Primero:#Si no hay nadie
                    print("El primero en aculatarse: {}".format(Entidad))
                    Destino.horas_eventos["Carga"] = Reloj + Destino.duracion_atencion()
                    Destino.hora_proximo_evento()#Actualizamos la hora del evento siguiente
                    Destino.disponibilidad = False
                    print("Camion termino de aculatarse y comienza atencion. Fin atencion: {}".format(Destino.horas_eventos["Carga"]))
                else: #Si no hay disponibilidad, se espera a que termine la atencion en el lugar de aculatamiento
                    print("{} No era el primero: {} o la pala no estaba disponible: {}".format(Entidad, Primero, Destino.disponibilidad))
                    pass
                Destino.orden_llegada.insert(0, Primero)
                
        #elif Entidad.estado == "Cargando":
        #    Destino = D_Entidades[Entidad.destino]
        #    Entidad.cargar(Destino.capacidad, Entidad.OD[1])
        #    Entidad.actualizar_estado("Viajando", Reloj + MTV[Entidad.origen][Entidad.destino])
    
    if Entidad.__class__.__name__ == "Pala":
        
        print(Entidad, Entidad.horas_eventos, Entidad.nombre_proximo_evento())
        if Entidad.nombre_proximo_evento() == "Carga":#Finaliza la carga del camion en atencion
            #Se busca el camion que termino la atencion
            print("Orden de llegada: {}".format(Entidad.orden_llegada))
            Camion = Entidad.orden_llegada.popleft()
            #Despacho al camion:
            i = Entidad.espacios_aculatamiento.index(Camion)
            Entidad.espacios_aculatamiento[i] = 0 
            
            #El camion sale del lugar de aculatamiento
            #Camion.cargar(Entidad.capacidad, Camion.OD[1])
            #Camion.actualizar_estado("Viajando", Reloj + MTV[Camion.origen][Camion.destino])
            
            Camion.nuevo_viaje("Viajando", MTV[D_Entidades[Camion.OD[0]].ubicacion][Camion.OD[1]]+Reloj)
            print("Camion despachado: {}, llega a las: {} a {}".format(Camion, Camion.hora_siguiente, Camion.destino))
            Entidad.disponibilidad = True
            Entidad.horas_eventos["Carga"] = Reloj*99
            Entidad.hora_proximo_evento()
            print("Estado del orden de llegada {}".format(Entidad.orden_llegada))
            #Reviso si quedan camiones por atender
            if len(Entidad.orden_llegada) > 0:
                Camion = Entidad.orden_llegada.pop()
                print(Camion, Camion.estado)
                if Camion.estado == "Cargando":
                    Entidad.disponibilidad = False
                    
                   
                    Entidad.horas_eventos["Carga"] = Reloj + Entidad.duracion_atencion()
                    print("Comienzo a atender a: {} y termino a las {}".format(Camion, Entidad.horas_eventos["Carga"]))
                    Entidad.hora_proximo_evento()
                    print("Proximo evento: {}".format(Entidad.hora_siguiente))
                Entidad.orden_llegada.append(Camion)
            
            print(Entidad.disponibilidad)
            if len(Entidad.fila) > 0:
                Camion = Entidad.fila.popleft()
                Entidad.orden_llegada.append(Camion)
                i = Entidad.espacios_aculatamiento.index(0)
                Entidad.espacios_aculatamiento[i] = Camion
                Camion.actualizar_estado("Aculatandose", Reloj + Destino.tiempo_aculatado)
                print("Lugar aculatado: {}".format(Entidad.espacios_aculatamiento))
                print("Fila: {}".format(Entidad.fila))     
    

    try:        
        print(Entidad, Entidad.hora_siguiente, Entidad.destino, Entidad.estado)  
        print('\n')
    except:
        print(Entidad, Entidad.hora_siguiente)
        print('\n')
    print('------------------------------------------------------------')              
    trucks += 1    
    L_Entidades.sort(key=lambda x: x.hora_siguiente, reverse=False)
   

        

------------------------------------------------------------
C0 0
C0 CS03 CS03 Descargando 0 0
El resultado del dinamico fue: ('PA11', 'S1')
C0 23.741270811406615 PA11 Viajando


------------------------------------------------------------
------------------------------------------------------------
C1 0
C1 CS03 CS03 Descargando 0 0
El resultado del dinamico fue: ('PA12', 'S2')
C1 21.892935414714852 PA12 Viajando


------------------------------------------------------------
------------------------------------------------------------
C2 0
C2 CS03 CS03 Descargando 0 0
El resultado del dinamico fue: ('PA12', 'CS03')
C2 21.892935414714852 PA12 Viajando


------------------------------------------------------------
------------------------------------------------------------
C3 0
C3 CS03 CS03 Descargando 0 0
El resultado del dinamico fue: ('PA10', 'S1')
C3 37.453769680793656 PA10 Viajando


------------------------------------------------------------
--------------------------------------

C6 PA11 CS03 Viajando 122.80397267307688 0
C6 123.97063933974356 CS03 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C6 123.97063933974356
C6 PA11 CS03 Aculatandose 123.97063933974356 0
C6 125.60180475625542 CS03 Descargando


------------------------------------------------------------
------------------------------------------------------------
PA13 124.28510956325357
PA13 {'Recuperacion': 99999999, 'Caida': 9999999, 'Inicio Movimiento': 99999999, 'Fin Movimiento': 99999999, 'Carga': 124.28510956325357} Carga
Orden de llegada: deque([C7])
Camion despachado: C7, llega a las: 163.73336764603602 a S1
Estado del orden de llegada deque([])
True
PA13 12304.225846762103


------------------------------------------------------------
------------------------------------------------------------
C6 125.60180475625542
C6 PA11 CS03 Descargando 125.60180475625542 0
No habia fila
El lugar de aculatado en CS03 

PA11 {'Recuperacion': 99999999, 'Caida': 9999999, 'Inicio Movimiento': 99999999, 'Fin Movimiento': 99999999, 'Carga': 224.3563592827582} Carga
Orden de llegada: deque([C6])
Camion despachado: C6, llega a las: 262.47029159697723 a S2
Estado del orden de llegada deque([])
True
PA11 22211.27956899306


------------------------------------------------------------
------------------------------------------------------------
PA20 224.5029910623996
PA20 {'Recuperacion': 99999999, 'Caida': 9999999, 'Inicio Movimiento': 99999999, 'Fin Movimiento': 99999999, 'Carga': 224.5029910623996} Carga
Orden de llegada: deque([C2])
Camion despachado: C2, llega a las: 261.7229277290328 a S2
Estado del orden de llegada deque([])
True
PA20 22225.79611517756


------------------------------------------------------------
------------------------------------------------------------
C7 225.4154424151293
C7 PA10 S2 Aculatandose 225.4154424151293 0
C7 226.58210908179595 S2 Descargando


----------------------------

C6 460.6642713763823 PA12 Viajando


------------------------------------------------------------
------------------------------------------------------------
C7 425.83797728121436
C7 CASINO PA11 Viajando 425.83797728121436 1
No hay fila, C7 se aculata deque([])
Camion aculatandose en PA11:[C7, 0]
C7 427.00464394788105 PA11 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C2 426.8900581733762
C2 CS03 CASINO Almorzando 426.8900581733762 1
El camion C2 termino de almorzar.
El resultado del dinamico fue: ('PA13', 'S1')
C2 452.63714977432375 PA13 Viajando


------------------------------------------------------------
------------------------------------------------------------
C7 427.00464394788105
C7 CASINO PA11 Aculatandose 427.00464394788105 1
El primero en aculatarse: C7
Camion termino de aculatarse y comienza atencion. Fin atencion: 428.5330899997826
C7 42273.45975084022 PA11 Cargando


-----------

C5 PA12 S1 Viajando 504.5893828344597 1
C5 505.75604950112637 S1 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C5 505.75604950112637
C5 PA12 S1 Aculatandose 505.75604950112637 1
C5 506.92271616779306 S1 Descargando


------------------------------------------------------------
------------------------------------------------------------
C5 506.92271616779306
C5 PA12 S1 Descargando 506.92271616779306 1
No habia fila
El lugar de aculatado en S1 es [C5, 0, 0, 0, 0]
El lugar de aculatado en S1 quedo [0, 0, 0, 0, 0]
El resultado del dinamico fue: ('PA21', 'S2')
C5 542.1079311901561 PA21 Viajando


------------------------------------------------------------
------------------------------------------------------------
C3 509.5710333369927
C3 PA13 S1 Viajando 509.5710333369927 1
C3 510.73770000365937 S1 Aculatandose


------------------------------------------------------------
-------------------------

Camion despachado: C5, llega a las: 643.6999900393306 a S2
Estado del orden de llegada deque([])
True
PA8 60577.52947827314


------------------------------------------------------------
------------------------------------------------------------
C6 611.9608322091477
C6 CS03 PA10 Aculatandose 611.9608322091477 1
El primero en aculatarse: C6
Camion termino de aculatarse y comienza atencion. Fin atencion: 613.8874719024968
C6 60584.122388705626 PA10 Cargando


------------------------------------------------------------
------------------------------------------------------------
PA10 613.8874719024968
PA10 {'Recuperacion': 99999999, 'Caida': 9999999, 'Inicio Movimiento': 99999999, 'Fin Movimiento': 99999999, 'Carga': 613.8874719024968} Carga
Orden de llegada: deque([C6])
Camion despachado: C6, llega a las: 634.8567502429338 a S2
Estado del orden de llegada deque([])
True
PA10 60774.85971834719


------------------------------------------------------------
------------------------------

C5 681.615865322914 PA15 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C5 681.615865322914
C5 S2 PA15 Aculatandose 681.615865322914 1
El primero en aculatarse: C5
Camion termino de aculatarse y comienza atencion. Fin atencion: 686.9187939062869
C5 67479.9706669685 PA15 Cargando


------------------------------------------------------------
------------------------------------------------------------
C3 684.0272680080473
C3 S1 PA11 Viajando 684.0272680080473 1
No hay fila, C3 se aculata deque([])
Camion aculatandose en PA11:[C3, 0]
C3 685.1939346747139 PA11 Aculatandose


------------------------------------------------------------
------------------------------------------------------------
C0 684.774702800037
C0 PA10 CS03 Viajando 684.774702800037 1
C0 685.9413694667036 CS03 Aculatandose


------------------------------------------------------------
----------------------------------------------

In [12]:
origen = "CASINO"
destino = "PA8"
print(MTV["CASINO"][D_Entidades[destino].ubicacion])
D_Entidades[destino].ubicacion

25.674369589637212


'PO5'